### ITCR - Escuela de Computación
### Curso IC-6200 - Inteligencia Artificial
### Aprendizaje supervisado

### Redes de memoria de corto y largo plazo con PyTorch 
### (Long-Short Term Memory Networks-LSTM)

**Profesora: María Auxiliadora Mora**

**Estudiantes:**
- Jonathan Quesada Salas
- Rodolfo Cruz Vega

## Introducción

La clasificación de textos y el reconocimiento de entidades nombradas (Named Entity Recognition o NER por sus siglas en inglés) son técnicas fundamentales que constituyen el primer paso en muchas tareas de Procesamiento de lenguaje natural (NLP). NER, es un área de investigación relacionada a la extracción de información, que permite localizar y clasificar nombres de entidades que se encuentran en texto libre, en categorías comunmente organizaciones, lugares, tiempo, personas, entre otros. Ejemplo:

- El fundador de [Microsoft Corporation] (organización), [Bill Gates] (persona), comentó que se abrirán 1000 puestos de trabajo en la [Región Chorotega] (lugar) a partir del año 2022 (fecha).  

La clasificación de textos permite categorizar el contenido asociando este a un conjunto de etiquetas predefinidas o clases. Su uso más popular es el análisis de sentimientos. Ejemplo:

- En mi opinión, la película fue muy buena porque pudo dar a conocer a los espectadores cómo puede afectar una situación traumática a la mente humana. (Clase = 5 o excelente). 

Las redes neuronales recurrentes o RNN (Rumelhart et al., 1986, como se citó en LeCun et al., 2015) son una familia de redes neuronales para el procesamiento de secuencias de datos, las cuales en un tiempo t, reciben el estado anterior, es decir, su salida en el tiempo t podría usarse como insumo del procesamiento de la siguiente entrada, de modo que la información pueda propagarse a medida que la red pasa por la secuencia de entrada. Las redes Long Short-Term Memory (LSTM) son un tipo de red neuronal recurrente capaz de aprender dependencias a largo plazo.

El siguiente ejemplo implementa NER con una LSTM para etiquetar el rol que juegan las palabras en las oraciones. 


## Ejemplo

El sistema implementado en el código adjunto soluciona el problema de estimar el rol de una palabra en una frase, por ejempo roles como determinante (DET), nombre (NN) y verbo (V). 
Ejemplo para la frase:

- "El perro come manzana" la salida deberá ser: ["DET", "NN", "V", "NN"]). 

Este proceso se conoce en el procesamiento de lenguaje natural como "part of speech tagging (POS)".

Este es un ejemplo simple con datos introducidos en el código basado en [1].

Se realizarán los siguientes pasos

   * Definición de los ejemplos (codificados) 
   * Preprocesamiento de las palabras a clasificar
   * Definición del modelo
   * Instanciación del modelo, definición de la función de pérdida y del optimizador  
   * Entrenamiento de la red
   * Pruebas del modelo resultante con unos cuantos ejemplos.


In [ ]:
# Bibliotecas requeridas

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv

torch.manual_seed(1)

In [ ]:
# Funciones utilitarias

def max_values(x):
    """
    Retorna el valor máximo y en índice o la posición del valor en un vector x.
    Parámetros: 
        x: vector con los datos. 
    Salida: 
        out: valor 
        inds: índice
    """
    out, inds = torch.max(x,dim=1)   
    return out, inds
    

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer los datos del archivo CSV
data = pd.read_csv('datos.csv', encoding="utf-8")

# Convertir las columnas en listas de Python
sentences = data['sentence'].tolist()
tags = data['tags'].tolist()

# Dividir los datos en conjuntos de entrenamiento y prueba
train_sentences, test_sentences, train_tags, test_tags = train_test_split(sentences, tags, test_size=0.2, random_state=42)

# Mostrar los tamaños de los conjuntos de datos resultantes
print("Tamaño del conjunto de entrenamiento:", len(train_sentences))
print("Tamaño del conjunto de prueba:", len(test_sentences))

print(train_sentences[0])
print(train_tags[0])

Tamaño del conjunto de entrenamiento: 40
Tamaño del conjunto de prueba: 10
El agua fría refresca
DET NN ADJ V


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import csv

# Leer los datos del archivo CSV
data = pd.read_csv('datos.csv', encoding="utf-8")

# Convertir las columnas en listas de Python
sentences = data['sentence'].tolist()
tags = data['tags'].tolist()

# Dividir los datos en conjuntos de entrenamiento y prueba
train_sentences, test_sentences, train_tags, test_tags = train_test_split(sentences, tags, test_size=0.2, random_state=42)

# Frases de entrenamiento 
# El modelo solo va a aprender a identificar DET, NN, V 
training_data = []
for i in range(len(train_sentences)):
    training_data.append((train_sentences[i].split(), train_tags[i].split()))

# Datos de prueba
test_data = []
for i in range(len(test_sentences)):
    test_data.append((test_sentences[i].split(), test_tags[i].split()))

# Diccionario las palabras
word_to_ix = {}
for sent, tags in training_data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print("Diccionario", word_to_ix)

# Asignar índices a las etiquetas
tag_to_ix = {"DET": 0, "NN": 1, "V": 2, "ADJ": 3}


# Preparación de los datos 
def prepare_sequence(seq, to_ix):
    """
    Retorna un tensor con los indices del diccionario para cada palabras en una oración.
    Parámetros:
       seq: oración
       to_ix: diccionario de palabras.
    """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)



Diccionario {'El': 0, 'agua': 1, 'fría': 2, 'refresca': 3, 'La': 4, 'flor': 5, 'marchita': 6, 'está': 7, 'triste': 8, 'lluvia': 9, 'suave': 10, 'cae': 11, 'sobre': 12, 'el': 13, 'suelo': 14, 'árbol': 15, 'frondoso': 16, 'da': 17, 'sombra': 18, 'hombre': 19, 'cansado': 20, 'corre': 21, 'rápidamente': 22, 'casa': 23, 'grande': 24, 'es': 25, 'hermosa': 26, 'serpiente': 27, 'sigilosa': 28, 'se': 29, 'arrastra': 30, 'silenciosamente': 31, 'sol': 32, 'caliente': 33, 'calienta': 34, 'día': 35, 'de': 36, 'verano': 37, 'puerta': 38, 'abierta': 39, 'par': 40, 'en': 41, 'invita': 42, 'a': 43, 'entrar': 44, 'mujer': 45, 'elegante': 46, 'camina': 47, 'lentamente': 48, 'brillante': 49, 'ilumina': 50, 'perro': 51, 'ruidoso': 52, 'ladra': 53, 'sin': 54, 'parar': 55, 'comida': 56, 'deliciosa': 57, 'hace': 58, 'la': 59, 'boca': 60, 'niño': 61, 'estudioso': 62, 'concentra': 63, 'sus': 64, 'tareas': 65, 'bebé': 66, 'travieso': 67, 'gatea': 68, 'gato': 69, 'negro': 70, 'duerme': 71, 'plácidamente': 72, 'em

In [ ]:
# Ejemplo de procesamiento de una oración
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(training_data[0][0])                          
print(inputs)

print(training_data)

['El', 'agua', 'fría', 'refresca']
tensor([0, 1, 2, 3])
[(['El', 'agua', 'fría', 'refresca'], ['DET', 'NN', 'ADJ', 'V']), (['La', 'flor', 'marchita', 'está', 'triste'], ['DET', 'NN', 'ADJ', 'V', 'ADJ']), (['La', 'lluvia', 'suave', 'cae', 'sobre', 'el', 'suelo'], ['DET', 'NN', 'ADJ', 'V', 'NN', 'DET', 'NN']), (['El', 'árbol', 'frondoso', 'da', 'sombra'], ['DET', 'NN', 'ADJ', 'V', 'ADJ']), (['El', 'hombre', 'cansado', 'corre', 'rápidamente'], ['DET', 'NN', 'ADJ', 'V', 'ADJ']), (['La', 'casa', 'grande', 'es', 'hermosa'], ['DET', 'NN', 'ADJ', 'V', 'ADJ']), (['La', 'serpiente', 'sigilosa', 'se', 'arrastra', 'silenciosamente'], ['DET', 'NN', 'ADJ', 'DET', 'V', 'ADJ']), (['El', 'sol', 'caliente', 'calienta', 'el', 'día', 'de', 'verano'], ['DET', 'NN', 'ADJ', 'V', 'DET', 'NN', 'DET', 'NN']), (['La', 'puerta', 'abierta', 'de', 'par', 'en', 'par', 'invita', 'a', 'entrar'], ['DET', 'NN', 'ADJ', 'V', 'NN', 'V', 'NN', 'V', 'DET', 'V']), (['La', 'mujer', 'elegante', 'camina', 'lentamente'], ['DET', 

In [ ]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    """
    Clase para aplicar POST a oraciones en español. 
    """
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        """
        Inicialización de la clase.
        Parámetros:
           embedding_dim: dimesionalidad del vector de palabras. 
           hidden_dim: dimensión de la capa oculta de la red. 
           vocab_size: tamaño del vocabulario.  
           tagset_size: número de clases.
        """
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. Ejemplos en [3] y [4]
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera los estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de clases
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pasada hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

        # Se utiliza softmax para devolver la probabilidad de cada etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [ ]:
# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Generalmente utilizada en problemas de clasificacion con múltiples clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [ ]:
# Entrenamiento del modelo 

# Valores antes de entrenar
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

# Épocas de entrenamiento
for epoch in range(200):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

['El', 'agua', 'fría', 'refresca']
tensor([[-1.1945, -1.2007, -1.7257, -1.5228],
        [-1.1901, -1.2156, -1.7265, -1.5079],
        [-1.2055, -1.1415, -1.7726, -1.5549],
        [-1.1728, -1.2043, -1.7058, -1.5655]])
Resultados luego del entrenamiento para la primera frase
tensor([[-9.4977e-04, -6.9671e+00, -1.1981e+01, -1.4436e+01],
        [-1.3121e+01, -2.7403e-03, -1.8587e+01, -5.9018e+00],
        [-7.0993e+00, -7.7049e+00, -7.4247e+00, -1.8744e-03],
        [-8.2136e+00, -1.4277e+01, -4.1257e-03, -5.5609e+00]])


In [ ]:
# Uso del modelo generado

def test_examples(test_data):

   with torch.no_grad():
      inputs = prepare_sequence(test_data, word_to_ix)
      tag_scores = model(inputs)
    
   # Se imprime los resultados de las frases
   print("FRASE") 
   print("La frase original", test_data)    
   print("La frase original preprocesada", inputs)
   print("Salida del modelo", tag_scores)
   print("Valores máximos e índices", max_values(tag_scores))    
    
print("Clases")
print(tag_to_ix)

#Frase 1
# Las palabras en una oración se pueden etiquetar de tres formas.
# La primera oración tiene 3 palabras "El perro juega"
# por eso el tensor de salida tiene 3 elementos. 
# Cada elemento es un vector de probabilidad de estar asociada a una clase. 
# Es decir hay que calcular la posición del valor máximo. 
#   Ejemplo 1: "El perro juega" ["DET", "NN", "V"]
# Ejemplo: salida 0, 1, 2 con {"DET": 0, "NN": 1, "V": 2} => DET, NN, V 
test_examples(test_data[0][0])

Clases
{'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3}
FRASE
La frase original ['El', 'pájaro', 'alegre', 'canta', 'melodías', 'hermosas']
La frase original preprocesada tensor([  0, 156, 157, 158, 159, 160])
Salida del modelo tensor([[-9.4977e-04, -6.9671e+00, -1.1981e+01, -1.4436e+01],
        [-1.2067e+01, -1.1533e-03, -1.8914e+01, -6.7707e+00],
        [-9.3166e+00, -4.7764e+00, -7.2982e+00, -9.2352e-03],
        [-1.2215e+01, -9.4336e+00, -1.4378e-01, -2.0111e+00],
        [-6.3779e+00, -5.5018e+00, -3.4360e+00, -3.8710e-02],
        [-6.1086e+00, -9.4115e+00, -5.6043e+00, -6.0057e-03]])
Valores máximos e índices (tensor([-0.0009, -0.0012, -0.0092, -0.1438, -0.0387, -0.0060]), tensor([0, 1, 3, 2, 3, 3]))


In [ ]:
#Frase 2
test_examples(test_data[0][0])
print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['El', 'pájaro', 'alegre', 'canta', 'melodías', 'hermosas']
La frase original preprocesada tensor([  0, 156, 157, 158, 159, 160])
Salida del modelo tensor([[-9.4977e-04, -6.9671e+00, -1.1981e+01, -1.4436e+01],
        [-1.2067e+01, -1.1533e-03, -1.8914e+01, -6.7707e+00],
        [-9.3166e+00, -4.7764e+00, -7.2982e+00, -9.2352e-03],
        [-1.2215e+01, -9.4336e+00, -1.4378e-01, -2.0111e+00],
        [-6.3779e+00, -5.5018e+00, -3.4360e+00, -3.8710e-02],
        [-6.1086e+00, -9.4115e+00, -5.6043e+00, -6.0057e-03]])
Valores máximos e índices (tensor([-0.0009, -0.0012, -0.0092, -0.1438, -0.0387, -0.0060]), tensor([0, 1, 3, 2, 3, 3]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3}


In [ ]:
# Otra prueba
test_examples(test_data[2][0])
print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['La', 'música', 'suave', 'suena', 'relajante']
La frase original preprocesada tensor([  4, 165,  10, 121, 166])
Salida del modelo tensor([[-5.7335e-04, -7.4752e+00, -1.2082e+01, -1.4449e+01],
        [-1.3905e+01, -2.2060e-03, -1.9453e+01, -6.1181e+00],
        [-5.5906e+00, -4.3287e+00, -4.8420e+00, -2.5122e-02],
        [-7.7758e+00, -1.1870e+01, -6.6100e-03, -5.0895e+00],
        [-7.4717e+00, -1.3817e+01, -4.7315e-03, -5.4846e+00]])
Valores máximos e índices (tensor([-0.0006, -0.0022, -0.0251, -0.0066, -0.0047]), tensor([0, 1, 3, 2, 2]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3}


In [ ]:
true_tags = [[tag_to_ix[tag] for tag in tags] for words, tags in test_data]
print(true_tags)

[[0, 1, 3, 2, 1, 3], [0, 1, 3, 2, 0, 0, 1, 3], [0, 1, 3, 2, 3], [0, 1, 3, 2, 1, 0, 3], [0, 1, 3, 2, 0, 1], [0, 1, 3, 2, 0, 1, 3], [0, 1, 3, 2, 1, 0, 3], [0, 1, 3, 2, 0, 1, 3], [0, 1, 3, 2, 0, 0, 1, 3], [0, 1, 3, 2, 0, 3]]


In [ ]:
true_labels = []
# Se recorre todos los datos de test_data
for sentence, labels in test_data:
   # print("sentence", len(sentence))
   # print("labels", len(labels))
    true_labels.extend(labels)

# Aquí es pura magia, se asigna el tag a cada sentencia para que puedan ajustarse a los valores de los tensores
true_labels = [tag_to_ix[tag] for sentence in test_data for tag in sentence[1]]
print(true_labels)    

[0, 1, 3, 2, 1, 3, 0, 1, 3, 2, 0, 0, 1, 3, 0, 1, 3, 2, 3, 0, 1, 3, 2, 1, 0, 3, 0, 1, 3, 2, 0, 1, 0, 1, 3, 2, 0, 1, 3, 0, 1, 3, 2, 1, 0, 3, 0, 1, 3, 2, 0, 1, 3, 0, 1, 3, 2, 0, 0, 1, 3, 0, 1, 3, 2, 0, 3]


In [ ]:
predicted_labels = []
# Se recore cada valor de test_data
for sentence, labels in test_data:
   #print("sentence", len(sentence))
    #print("labels", len(labels))

    # Aquí es pura magia se retornan los valores maximos de los tensores que se tuvieron en las oraciones
    predicted_labels.extend(max_values(model(prepare_sequence(sentence, word_to_ix)))[1])

print(predicted_labels)

[tensor(0), tensor(1), tensor(3), tensor(2), tensor(3), tensor(3), tensor(0), tensor(1), tensor(3), tensor(2), tensor(2), tensor(0), tensor(1), tensor(3), tensor(0), tensor(1), tensor(3), tensor(2), tensor(2), tensor(0), tensor(1), tensor(3), tensor(2), tensor(0), tensor(0), tensor(2), tensor(0), tensor(1), tensor(3), tensor(2), tensor(0), tensor(0), tensor(0), tensor(1), tensor(3), tensor(3), tensor(3), tensor(2), tensor(2), tensor(0), tensor(1), tensor(3), tensor(2), tensor(3), tensor(0), tensor(3), tensor(0), tensor(1), tensor(3), tensor(2), tensor(0), tensor(0), tensor(3), tensor(0), tensor(1), tensor(3), tensor(2), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(1), tensor(3), tensor(2), tensor(0), tensor(1)]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
# Se usa precision_recall_fscore_support para determinar la precision, exhaustividad y F1 en base a al average, true_labels y predicted_labels
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
# Se imprimen los resultados
print("Precisión:", precision)
print("Exhaustividad:", recall)
print("F1:", f1)

Precisión: 0.7880709671754448
Exhaustividad: 0.7761194029850746
F1: 0.7730661055167556


## Conclusiones
1. Se logró agregar la nueva clase de etiqueta y generar un conjunto de datos más grande, lo que permitió mejorar el desempeño del modelo. La división del conjunto de datos en entrenamiento y prueba permitió una mejor evaluación del modelo.

2. Es importante considerar al crear el conjunto de datos no contenga error en la asignacion de etiquetas en cuanto a el tipo y la cantidad ya que estos errores ca

3. El modelo de clasificación alcanzó una precisión promedio de 78.8%, lo que indica que en promedio es capaz de clasificar correctamente el 78.8% de las palabras en las frases de prueba.

4. La exhaustividad promedio del modelo fue del 77.6%, lo que significa que el modelo identificó correctamente el 77.6% de las palabras que realmente pertenecen a cada clase.

5. El valor de F1, que combina la precisión y la exhaustividad, fue de 0.773, lo que sugiere que el modelo es razonablemente bueno para clasificar las palabras en las frases de prueba.



# Referencias 

[1] Guthrie, R. (2017). Tutorial. Sequence Models and Long-Short Term Memory Networks. Recuperado de https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

[2] LeCun,Y., Bengio, Y.,  & Hinton, G. (2015). Deep learning. Nature, 521(7553):436.

[3] Brownlee, J. (2017). What Are Word Embeddings for Text?. Recuperado de https://machinelearningmastery.com/what-are-word-embeddings/

[4] Bishop, C (2006). Pattern Recognition and Machine Learning. Springer.